In [1]:
import neuroglancer
import numpy as np
import os
import sys
import navis

# add path to ac_ngl to sys.path
sys.path.append('/home/wanqing.yu/AC_Project/ac_visualization/')
from ac_ngl import *

/home/wanqing.yu/anaconda3/envs/AC/lib/python3.10/site-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [2]:
# source_path = '/ACdata/Users/wanqing/exaSPIM/forOlga/Result of tile_x_0002_y_0001_z_0000_ch_488_mip2-8bit-zcut.tif'
source_path = '/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.006_2023-09-22_14-52-45/exaSPIM/mip0/test.tif'

# out_path = '/ACdata/Users/wanqing/exaSPIM/precomputed'
out_path = '/ACdata/Users/wanqing/exaSPIM/test/precomputed/'

Generate ExASPIM precomputed skeletons

In [3]:
# s_range = [53,55]#[52,54]
indir = '/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.006_2023-09-22_14-52-45/exaSPIM/mip2/x2y1z0/skeleton/skeletons_10' #'/home/wanqing.yu/EM Connectome/axon_analysis/skeletons/skel_axon/' #/home/wanqing.yu/EM Connectome/axon_analysis/skeletons/rid_sid_cleaned/' #rid_only/' # '/ACdata/Users/connorl/Skeletons/For_Kevin/S32_Pos52,53,54_MIP0/' ###skeletons
outdir = '/ACdata/Users/wanqing/Neuroglancer/ExASPIM/' #skel_axon/'
precomputed_image = '/ACdata/Users/wanqing/exaSPIM/precomputed/'

mip = 0
skel_trans = [0,0,0] #[23000,0,0]
scale_im = [4000,2961,2962] # nm
dim_im = neuroglancer.CoordinateSpace(
                names=['z', 'y', 'x', 't'],
                units='nm',
                scales=scale_im,
            )

dim_seg = neuroglancer.CoordinateSpace(
                names=['z', 'y', 'x', 't'],
                units='nm',
                scales=scale_im,
            )

In [4]:
###Create precomputed volume for skeletons
pos_dir = indir
os.makedirs(outdir, exist_ok=True)
generate_ngl_segmentation_empty([288,288,21586], pos_dir, outdir, [512, 512, 64], [406, 406, 1997.72])
generate_ngl_skeletons(pos_dir, outdir,match_fname=False)

Generate neuroglancer link

In [4]:
###Load zarr and skeletons into neuroglancer instance
ip = 'localhost' 
port = 9998
neuroglancer.set_server_bind_address(bind_address=ip,bind_port=port)
viewer=neuroglancer.Viewer()

rotation_matrix_skeletons = np.array([[0, 0, 1, 0],
                                      [0, 1, 0, 0],
                                      [1, 0, 0, 0]])

#translation_vector_skeletons = [0, 0, 340]  # Adjust these values as needed

# Combine rotation and translation
#transformation_matrix_skeletons = rotation_matrix_skeletons + np.array([translation_vector_skeletons])


#load image and skeletons
###load image and skeletons
tr_im = neuroglancer.CoordinateSpaceTransform(input_dimensions = dim_im, output_dimensions =  dim_im) 
tr_seg = neuroglancer.CoordinateSpaceTransform(input_dimensions = dim_seg, output_dimensions =  dim_seg,
                                                  matrix = rotation_matrix_skeletons)

with viewer.txn() as s:
    s.layers['ExASPIM'] = neuroglancer.ImageLayer(source=['precomputed://http://bigkahuna.corp.alleninstitute.org/' + precomputed_image])
    # s.layers['ExASPIM'].layer.source[0].transform  = tr_im

with viewer.txn() as s:
    s.layers['skeletons'] = neuroglancer.SegmentationLayer(source=['precomputed://http://bigkahuna.corp.alleninstitute.org/' + outdir +'skeletons/'])
    s.layers['skeletons'].layer.source[0].transform  = tr_seg
    
print(viewer)

http://localhost:9998/v/69b28b2a0b0bd30c5f72893dbb559822b0c66a45/


In [6]:
###Create shareable neuroglancer link
view = s.to_json()
make_neuroglancer_url_vneurodata(view)

http://bigkahuna.corp.alleninstitute.org/neuroglancer/#!https://json.neurodata.io/v1?NGStateID=1t5Kss8qP_iqfQ


In [6]:
import copy
import time

def interpolate_to(final_state, frames_per_second=5, seconds=1):
    total_frames = int(round(seconds * frames_per_second))
    initial_state = viewer.state
    for frame_i in range(total_frames):
        t = frame_i / total_frames
        viewer.set_state(
            neuroglancer.ViewerState.interpolate(initial_state, final_state, t)
        )
        time.sleep(1 / frames_per_second)
    viewer.set_state(final_state)

def move_by(offset, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates += offset
    interpolate_to(final_state, **kwargs)

def move_to(final_voxel_coordinates, final_orientation, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates = final_voxel_coordinates
    final_state.crossSectionOrientation = final_orientation
    final_state.projectionOrientation = final_orientation
    interpolate_to(final_state, **kwargs)

def zoom_to(factor, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.navigation.zoomFactor = factor
    interpolate_to(final_state, **kwargs)

def do_move_by():
    move_by([100, 100, 100], seconds=1, frames_per_second=10)

def do_move_to(final_voxel_coordinates, final_orientation, seconds=1, frames_per_second=10):
    move_to(final_voxel_coordinates, final_orientation = final_orientation, seconds=seconds, frames_per_second=frames_per_second)

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

# def surface_normal_to_quaternion(normal):
#     normal = normalize(normal)
#     up_vector = np.array([0, 0, 1]) # need to update up_vector based on the previous step
#     axis = np.cross(up_vector, normal)
#     axis = normalize(axis)
#     angle = np.arccos(np.dot(up_vector, normal))
    
#     qx = axis[0] * np.sin(angle / 2)
#     qy = axis[1] * np.sin(angle / 2)
#     qz = axis[2] * np.sin(angle / 2)
#     qw = np.cos(angle / 2)
    
#     return [qx, qy, qz, qw]

def surface_normal_to_quaternion(normal, up_vector = np.array([0, 0, 1])):
    normal = normalize(normal)
    # up_vector = np.array([0, 0, 1])
    axis = np.cross(up_vector, normal)
    axis = normalize(axis)
    angle = np.arccos(np.dot(up_vector, normal))
    
    qx = axis[0] * np.sin(angle / 2)
    qy = axis[1] * np.sin(angle / 2)
    qz = axis[2] * np.sin(angle / 2)
    qw = np.cos(angle / 2)
    
    return [-qx, -qy, -qz, qw], axis


import math

def angle_between_vectors(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    cos_angle = np.clip(cos_angle, -1, 1)  # Ensure within valid range for arccos
    angle_radians = np.arccos(cos_angle)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

def flip_vectors_if_necessary(skel_dp_vect):
    """Flip the direction of vectors if the angle with the previous vector is > 150 degrees."""
    for i in range(1, len(skel_dp_vect)):
        angle = angle_between_vectors(skel_dp_vect[i], skel_dp_vect[i-1])
        if angle > 130:
            skel_dp_vect[i] = abs(skel_dp_vect[i]) * np.sign(skel_dp_vect[i-1])
    return skel_dp_vect


def quaternion_to_rotation_angle(q1, q2):
    # Normalize the quaternions
    q1 = q1 / np.linalg.norm(q1)
    q2 = q2 / np.linalg.norm(q2)
    
    # Compute the dot product
    dot_product = np.dot(q1, q2)
    
    # Ensure the dot product is within the valid range for arccos
    dot_product = np.clip(dot_product, -1.0, 1.0)
    
    # Calculate the angle in radians
    angle_rad = 2 * np.arccos(dot_product)
    
    # Convert the angle to degrees
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

def flip_quaternion_axis(quaternion):
    """Flip the axis of a quaternion by negating the x, y, and z components."""
    for i in range(1, len(quaternion)):
        angle = quaternion_to_rotation_angle(quaternion[i], quaternion[i-1])
        if angle > 110:
            quaternion[i] = np.array([-quaternion[i,0], -quaternion[i,1], -quaternion[i,2], quaternion[i,3]])
    return quaternion

def get_tangent_vector(skel_navis):
    skel_dp = skel_navis.nodes[['x', 'y', 'z']].diff().values
    skel_dp_vect = np.array([normalize(v) for v in skel_dp])
    # skel_dp_vect = flip_vectors_if_necessary(skel_dp_vect)
    return skel_dp_vect


## zoom in movement

In [24]:
# initial position
with viewer.txn() as s:
    s.voxel_coordinates[0] = 360
    s.voxel_coordinates[1] = 1342.8425
    s.voxel_coordinates[2] = 1880.6376
    s.cross_section_scale = 4

In [21]:
# zoom in 
final_state = copy.deepcopy(viewer.state)
final_state.cross_section_scale = 0.4
interpolate_to(final_state, frames_per_second=90, seconds=3)

In [15]:
# current position
with viewer.txn() as s:
    s.voxel_coordinates[0] = 253

In [16]:
## loading all skeletons 
skel_all_navis = navis.read_swc('/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.006_2023-09-22_14-52-45/exaSPIM/mip2/x2y1z0/skeleton/skeletons_10/')
skel_all_navis

Importing:   0%|          | 0/6004 [00:00<?, ?it/s]

,type,name,n_nodes,n_connectors,n_branches,n_leafs,cable_length,soma,units,created_at,origin
0,navis.TreeNeuron,3358,18,None,0,1,144.625000,None,1 dimensionless,2024-10-03 09:06:29.314355,/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.0...
1,navis.TreeNeuron,2518,22,None,0,1,174.097763,None,1 dimensionless,2024-10-03 09:06:29.306748,/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.0...
...,...,...,...,...,...,...,...,...,...,...,...
6002,navis.TreeNeuron,3547,17,None,0,1,141.712128,None,1 dimensionless,2024-10-03 09:06:32.691043,/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.0...
6003,navis.TreeNeuron,3935,15,None,0,1,134.598907,None,1 dimensionless,2024-10-03 09:06:32.683116,/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.0...


In [21]:
id = 1
#skel_navis = skel_all_navis[skel_all_navis.name == str(id+1)].copy() 
skel_navis = navis.read_swc('/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.006_2023-09-22_14-52-45/exaSPIM/mip2/x2y1z0/skeleton/skeletons_10/0002.swc')
skel_navis.nodes


,node_id,label,x,y,z,radius,parent_id,type
0,1,2,1743.849976,1924.849976,246.000000,3.2480,-1,root
1,2,2,1736.180054,1924.000000,249.000000,6.4960,1,slab
2,3,2,1727.880005,1923.000000,252.117996,6.4960,2,slab
3,4,2,1720.449951,1921.449951,255.000000,6.4960,3,slab
4,5,2,1713.199951,1920.800049,257.802002,6.4960,4,slab
...,...,...,...,...,...,...,...,...
144,145,2,900.346985,2617.649902,337.000000,7.2627,144,slab
145,146,2,898.057983,2624.939941,334.058014,6.4960,145,slab
146,147,2,894.570984,2632.429932,331.000000,6.4960,146,slab
147,148,2,891.000000,2639.800049,328.000000,6.4960,147,slab


In [22]:
# switch x and z in skel_navis.nodes
tmp_x = skel_navis.nodes.x
skel_navis.nodes.x = skel_navis.nodes.z
skel_navis.nodes.z = tmp_x
skel_navis.nodes

,node_id,label,x,y,z,radius,parent_id,type
0,1,2,246.000000,1924.849976,1743.849976,3.2480,-1,root
1,2,2,249.000000,1924.000000,1736.180054,6.4960,1,slab
2,3,2,252.117996,1923.000000,1727.880005,6.4960,2,slab
3,4,2,255.000000,1921.449951,1720.449951,6.4960,3,slab
4,5,2,257.802002,1920.800049,1713.199951,6.4960,4,slab
...,...,...,...,...,...,...,...,...
144,145,2,337.000000,2617.649902,900.346985,7.2627,144,slab
145,146,2,334.058014,2624.939941,898.057983,6.4960,145,slab
146,147,2,331.000000,2632.429932,894.570984,6.4960,146,slab
147,148,2,328.000000,2639.800049,891.000000,6.4960,147,slab


In [23]:
# rotate axis
root_pos = [skel_navis.nodes.x[0], skel_navis.nodes.y[0], skel_navis.nodes.z[0]]
root_pos

[246.0, 1924.85, 1743.85]

In [97]:
# make a cetain segment visible
with viewer.txn() as s:
  s.layers['segmentation'].segments = [id]
  s.voxel_coordinates = root_pos

In [99]:
# get tangent vector
skel_dp_vect = get_tangent_vector(skel_navis)
skel_dp_vect

array([[        nan,         nan,         nan],
       [ 0.3623406 , -0.10266022, -0.9263747 ],
       [ 0.3494495 , -0.11207502, -0.9302281 ],
       [ 0.3549807 , -0.19092183, -0.9151708 ],
       [ 0.3592426 , -0.08332349, -0.9295171 ],
       [ 0.3618551 , -0.09052593, -0.9278286 ],
       [ 0.08365501,  0.        , -0.99649477],
       [ 0.33849743,  0.22956957, -0.91253346],
       [ 0.36353612,  0.11209831, -0.92481107],
       [ 0.29045334,  0.1106489 , -0.95047027],
       [ 0.32399267,  0.        , -0.94605964],
       [ 0.3363469 ,  0.11211563, -0.93504053],
       [ 0.22121705,  0.11060853, -0.96893173],
       [ 0.2741185 ,  0.11044282, -0.9553331 ],
       [ 0.3825685 ,  0.0010842 , -0.92392653],
       [ 0.57688713,  0.11454342, -0.80875283],
       [ 0.23833399,  0.10393984, -0.9656052 ],
       [ 0.4385345 ,  0.14313717, -0.88724256],
       [ 0.45460057,  0.2034382 , -0.86715114],
       [ 0.45316687,  0.22658344, -0.8621483 ],
       [ 0.5026038 ,  0.11893097, -0.856

In [104]:
# get the dotprops
skel_dp = navis.make_dotprops(skel_navis, k=15)
skel_dp.vect

array([[-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804647,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804647,  0.94463724],
       [-0.32901055, -0.05613139,  0.9426565 ],
       [-0.32929978, -0.07197427,  0.9414783 ],
       [-0.3392978 , -0.09479237,  0.93589073],
       [-0.34856644, -0.10382613,  0.93151575],
       [-0.3609784 , -0.10769283,  0.92633516],
       [-0.37204918, -0.10738385,  0.92198056],
       [-0.37954226, -0.11325256,  0.91821647],
       [-0.38392726, -0.11648791,  0.91598606],
       [-0.38378623, -0.12054848,  0.91551965],
       [-0.38162053, -0.12466606,  0.9158734 ],
       [-0.38052663, -0.1311981 ,  0.91541606],
       [-0.3723666 , -0.14273407,  0.917

In [105]:
skel_dp.vect = flip_vectors_if_necessary(skel_dp.vect)
skel_dp.vect

array([[-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804647,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804646,  0.94463724],
       [-0.32590333, -0.03804647,  0.94463724],
       [-0.32901055, -0.05613139,  0.9426565 ],
       [-0.32929978, -0.07197427,  0.9414783 ],
       [-0.3392978 , -0.09479237,  0.93589073],
       [-0.34856644, -0.10382613,  0.93151575],
       [-0.3609784 , -0.10769283,  0.92633516],
       [-0.37204918, -0.10738385,  0.92198056],
       [-0.37954226, -0.11325256,  0.91821647],
       [-0.38392726, -0.11648791,  0.91598606],
       [-0.38378623, -0.12054848,  0.91551965],
       [-0.38162053, -0.12466606,  0.9158734 ],
       [-0.38052663, -0.1311981 ,  0.91541606],
       [-0.3723666 , -0.14273407,  0.917

In [106]:
# get the quaternion for each node
quaternions = []
for i in range(len(skel_dp.vect)):
    quaternion, tmp = surface_normal_to_quaternion(skel_dp.vect[i])
    quaternions.append(quaternion)
quaternions

[[-0.01929212010189862, 0.1652549382387834, -0.0, 0.986062178308077],
 [-0.01929212010189862, 0.1652549382387834, -0.0, 0.986062178308077],
 [-0.01929212010189862, 0.1652549382387834, -0.0, 0.986062178308077],
 [-0.01929212010189862, 0.1652549382387834, -0.0, 0.986062178308077],
 [-0.019292121965473814, 0.16525493802122673, -0.0, 0.986062178308077],
 [-0.01929212010189862, 0.1652549382387834, -0.0, 0.986062178308077],
 [-0.01929212010189862, 0.1652549382387834, -0.0, 0.986062178308077],
 [-0.01929212010189862, 0.1652549382387834, -0.0, 0.986062178308077],
 [-0.019292121965473814, 0.16525493802122673, -0.0, 0.986062178308077],
 [-0.02847690889330443, 0.1669155689127834, -0.0, 0.9855598705884916],
 [-0.0365254774537461, 0.1671129363305222, -0.0, 0.9852609583291965],
 [-0.048174573346255634, 0.1724350410590945, -0.0, 0.9838421454165575],
 [-0.05282536251816268, 0.1773460020418473, -0.0, 0.9827298085611313],
 [-0.0548663306024457, 0.18390787852692517, -0.0, 0.9814110137867563],
 [-0.054770

In [107]:
# flip the quaternion if necessary
fixed_quaternions = flip_quaternion_axis(np.array(quaternions))
fixed_quaternions

array([[-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.01929212,  0.16525494, -0.        ,  0.98606218],
       [-0.02847691,  0.16691557, -0.        ,  0.98555987],
       [-0.03652548,  0.16711294, -0.        ,  0.98526096],
       [-0.04817457,  0.17243504, -0.        ,  0.98384215],
       [-0.05282536,  0.177346  , -0.        ,  0.98272981],
       [-0.05486633,  0.18390788, -0.        ,  0.98141101],
       [-0.05477084,  0.18976268, -0.        ,  0.98030112],
       [-0.0578208 ,  0.19377433, -0.        ,  0.97934073],
       [-0.05950722,  0.

In [77]:
skel_dp.vect[80:100]

array([[-0.15847406, -0.6734343 ,  0.7220611 ],
       [-0.13955134, -0.6905291 ,  0.7097147 ],
       [-0.13532847, -0.69528216,  0.70588166],
       [-0.14268145, -0.71143335,  0.68811667],
       [-0.06865818, -0.76575786,  0.63945365],
       [-0.00141493, -0.80536073,  0.59278333],
       [-0.00141493, -0.80536073,  0.59278333],
       [-0.10632256, -0.8425082 ,  0.52808654],
       [-0.10632256, -0.84250826,  0.52808654],
       [-0.24257307, -0.85580266,  0.45690277],
       [-0.33086267, -0.8528077 ,  0.40404075],
       [-0.3885394 , -0.83797073,  0.38319996],
       [-0.41391093, -0.82400525,  0.3869019 ],
       [-0.42545447, -0.80469   ,  0.41408026],
       [-0.41651306, -0.8019966 ,  0.4281569 ],
       [-0.41604054, -0.7966048 ,  0.4385556 ],
       [-0.41881597, -0.7890974 ,  0.44935337],
       [-0.41454852, -0.78648514,  0.45781076],
       [-0.4042348 , -0.7831932 ,  0.47244328],
       [-0.39174342, -0.7870705 ,  0.47650507]], dtype=float32)

In [78]:
quaternions[80:100]

[[-0.3628740833802325, 0.08539233455057432, -0.0, 0.9279173178742591],
 [-0.37342653051347807, 0.07546701897153457, -0.0, 0.9245849638384871],
 [-0.376419092972857, 0.07326553649147143, -0.0, 0.9235479563120198],
 [-0.3871845417499854, 0.07765175981541361, -0.0, 0.918726474434814],
 [-0.4228894864473265, 0.03791645410477436, -0.0, 0.9053876654563154],
 [-0.45122910544210937, 0.0007927587258289656, -0.0, 0.8924077912790298],
 [-0.45122910544004663, 0.000792759899882664, -0.0, 0.8924077912790298],
 [-0.48193131865398187, 0.060818605348333374, -0.0, 0.8740957049110343],
 [-0.481931349618364, 0.06081860499408683, -0.0, 0.8740956878635173],
 [-0.5013526803892058, 0.14210595966554507, -0.0, 0.8534936356494366],
 [-0.5089161761403911, 0.19744353304664675, -0.0, 0.837866562717895],
 [-0.503815311714159, 0.2336025506831712, -0.0, 0.8316249034250244],
 [-0.49475719383197214, 0.24852439823078992, -0.0, 0.8327370188934416],
 [-0.47849409899646306, 0.2529886679942237, -0.0, 0.8408567839365224],
 [-

In [112]:
# move along the axon
# set the initial state
with viewer.txn() as s:
    s.voxel_coordinates = [skel_navis.nodes['x'][0], skel_navis.nodes['y'][0], skel_navis.nodes['z'][0]]
    s.crossSectionOrientation = fixed_quaternions[0]

In [113]:
# do move to according to the list of positions and orientations in df
for i in range(len(skel_navis.nodes)-1):
    do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation = fixed_quaternions[i], seconds = 0.2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


In [80]:
# current position
with viewer.txn() as s:
    current_pos = [s.voxel_coordinates[2],s.voxel_coordinates[1],s.voxel_coordinates[0]]

In [81]:
# create a bbox around the current position
dimensions = [100, 100, 100]

# Calculate the min and max coordinates of the bounding box
min_coords = [current_pos[i] - dimensions[i] / 2 for i in range(3)]
max_coords = [current_pos[i] + dimensions[i] / 2 for i in range(3)]

# Create the bounding box as a dictionary or any other structure you prefer
bbox = {
    'min': min_coords,
    'max': max_coords
}
bbox

{'min': [1778.0, 1769.0, 457.0], 'max': [1878.0, 1869.0, 557.0]}

In [82]:
skel_id = []
for skel in skel_all_navis:
    skel.nodes["in_bbox"] = (
        skel.nodes["x"].between(bbox['min'][0], bbox['max'][0]) &
        skel.nodes["y"].between(bbox['min'][1], bbox['max'][1]) &
        skel.nodes["z"].between(bbox['min'][2], bbox['max'][2])
    )
    skel_inbox = navis.subset_neuron(skel, skel.nodes["in_bbox"])
    if skel_inbox.n_nodes > 0:
        skel_id.append(skel.name)
        print(skel.name)
skel_id = np.array(skel_id.astype(int))-1

4690
0109
5658


In [88]:
skel_id = np.array(skel_id.astype(int))-2
skel_id

array([4689,  108, 5657])

In [89]:
# make nearby segments visible
with viewer.txn() as s:
  s.layers['segmentation'].segments = skel_id
  # s.voxel_coordinates = root_pos

In [93]:
current_pos = [skel_navis.nodes['z'][0], skel_navis.nodes['y'][0], skel_navis.nodes['x'][0]]

# create a bbox around the current position
dimensions = [100, 100, 100]

# Calculate the min and max coordinates of the bounding box
min_coords = [current_pos[i] - dimensions[i] / 2 for i in range(3)]
max_coords = [current_pos[i] + dimensions[i] / 2 for i in range(3)]

# Create the bounding box as a dictionary or any other structure you prefer
bbox = {
    'min': min_coords,
    'max': max_coords
}

skel_id = []
for skel in skel_all_navis:
    skel.nodes["in_bbox"] = (
        skel.nodes["x"].between(bbox['min'][0], bbox['max'][0]) &
        skel.nodes["y"].between(bbox['min'][1], bbox['max'][1]) &
        skel.nodes["z"].between(bbox['min'][2], bbox['max'][2])
    )
    skel_inbox = navis.subset_neuron(skel, skel.nodes["in_bbox"])
    if skel_inbox.n_nodes > 0:
        skel_id.append(skel.name)
        print(skel.name)
skel_id = np.array(skel_id).astype(int)-1

3852
4690
0510
5544
0109
5658


In [91]:
skel_id = np.array(skel_id).astype(int)-1

In [94]:
skel_id

array([3851, 4689,  509, 5543,  108, 5657])

In [95]:
with viewer.txn() as s:
    s.voxel_coordinates = [skel_navis.nodes['x'][0], skel_navis.nodes['y'][0], skel_navis.nodes['z'][0]]
    s.crossSectionOrientation = fixed_quaternions[0]
    s.layers['segmentation'].segments = skel_id

In [45]:
# filter the skeletons that have a nodes that are within a certain distance from the current position
# get the distance of each node from the current position
dist = []
for i in range(len(skel_all_navis.nodes)):
    dist.append(math.sqrt((skel_all_navis.nodes['x'][i] - current_pos[2])**2 + (skel_all_navis.nodes['y'][i] - current_pos[1])**2 + (skel_all_navis.nodes['z'][i] - current_pos[0])**2))
dist

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3f1b271ad0>>
Traceback (most recent call last):
  File "/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3f1b271ad0>>
Traceback (most recent call last):
  File "/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [63]:
from ipywidgets import Button, VBox, Output
from IPython.display import display
import time

# Initialize widgets
btn_pause = Button(description='Pause')
btn_continue = Button(description='Continue', disabled=True)
btn_back = Button(description='Back', disabled=True)
btn_forth = Button(description='Forth', disabled=True)
output = Output()

# Global control variables
i = 20
paused = False

# def do_move_to(position, final_orientation, seconds):
#     # Placeholder for the actual movement function
#     with output:
#         print(f"Moving to {position} with orientation {final_orientation} in {seconds} seconds")

def update_buttons():
    global i
    btn_back.disabled = i <= 0
    btn_forth.disabled = i >= len(skel_navis.nodes) - 1
    btn_continue.disabled = not paused

def on_pause_clicked(b):
    global paused
    paused = True
    update_buttons()

def on_continue_clicked(b):
    global paused
    paused = False
    update_buttons()

def on_back_clicked(b):
    global i
    if i > 0:
        i -= 1
        do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

def on_forth_clicked(b):
    global i
    if i < len(skel_navis.nodes) - 1:
        i += 1
        do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

# Attach event handlers
btn_pause.on_click(on_pause_clicked)
btn_continue.on_click(on_continue_clicked)
btn_back.on_click(on_back_clicked)
btn_forth.on_click(on_forth_clicked)

# Display widgets
display(VBox([btn_pause, btn_continue, btn_back, btn_forth, output]))

# Initial execution
do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=0.2)

In [99]:
from ipywidgets import Button, VBox, Output
from IPython.display import display
import threading
import time

# Initialize widgets
btn_pause = Button(description='Pause')
btn_continue = Button(description='Continue', disabled=True)
btn_back = Button(description='Back', disabled=True)
btn_forth = Button(description='Forth', disabled=True)
output = Output()

# Global control variables
i = 0
paused = False
stop_thread = False

# def do_move_to(position, final_orientation, seconds):
#     # Placeholder for the actual movement function
#     with output:
#         print(f"Moving to {position} with orientation {final_orientation} in {seconds} seconds")

def update_buttons():
    global i
    btn_back.disabled = i <= 0
    btn_forth.disabled = i >= len(skel_navis.nodes) - 1
    btn_continue.disabled = not paused
    btn_pause.disabled = paused

def on_pause_clicked(b):
    global paused
    paused = True
    update_buttons()

def on_continue_clicked(b):
    global paused
    paused = False
    update_buttons()

def on_back_clicked(b):
    global i
    if i > 0:
        i -= 1
        do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

def on_forth_clicked(b):
    global i
    if i < len(skel_navis.nodes) - 1:
        i += 1
        do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

def loop_through_positions():
    global i, paused, stop_thread
    while not stop_thread:
        if not paused and i < len(skel_navis.nodes):
            do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=0.5)
            i += 1
            update_buttons()
        time.sleep(0.1)  # Adjust the sleep time as needed

# Attach event handlers
btn_pause.on_click(on_pause_clicked)
btn_continue.on_click(on_continue_clicked)
btn_back.on_click(on_back_clicked)
btn_forth.on_click(on_forth_clicked)

# Display widgets
display(VBox([btn_pause, btn_continue, btn_back, btn_forth, output]))

# Start the loop in a separate thread
thread = threading.Thread(target=loop_through_positions)
thread.start()

# Initial execution
# do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=0.2)

KeyError: 152

In [14]:
neuroglancer.stop()

In [ ]:
# loading all skeletons to 